# OneOffice Feature Infrastructure Setup

This notebook helps manage the infrastructure setup for the **OneOffice** feature.
It includes steps to:
1. Setup Secrets (Access Tokens)
2. Setup Cloud Scheduler Jobs


In [ ]:
import getpass
import subprocess

# Configuration Variables
PROJECT_ID = "sync-nhanhvn-project"
REGION_SCHEDULER = "us-central1"
REGION_RUN = "asia-southeast1"
SERVICE_ACCOUNT = f"{PROJECT_ID}@appspot.gserviceaccount.com"

print(f"Project: {PROJECT_ID}")
print(f"Service Account: {SERVICE_ACCOUNT}")

## 1. Setup Secrets
Securely adding OneOffice credentials to Google Secret Manager.

In [ ]:
print("Please provide OneOffice credentials:")
oneoffice_token = getpass.getpass("OneOffice Access Token: ")

def create_secret(name, value):
    try:
        subprocess.run(f"gcloud secrets describe {name} --project={PROJECT_ID}", shell=True, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        print(f"Secret '{name}' exists. Adding new version...")
        cmd = f"gcloud secrets versions add {name} --data-file=- --project={PROJECT_ID}"
    except subprocess.CalledProcessError:
        print(f"Creating secret '{name}'...")
        cmd = f"gcloud secrets create {name} --data-file=- --project={PROJECT_ID}"
    
    process = subprocess.run(cmd, input=value.encode(), shell=True, capture_output=True)
    if process.returncode == 0:
        print(f"Success: {name}")
    else:
        print(f"Error: {process.stderr.decode()}")

    # Grant Access
    subprocess.run(f"gcloud secrets add-iam-policy-binding {name} --member='serviceAccount:{SERVICE_ACCOUNT}' --role='roles/secretmanager.secretAccessor' --project={PROJECT_ID}", shell=True)

if oneoffice_token:
    create_secret("oneoffice-access-token", oneoffice_token)

## 2. Setup Cloud Scheduler
Sets up the scheduler for OneOffice Daily Sync.

In [ ]:
# OneOffice Daily Sync (Daily at 1:00 AM)
cmd = f"gcloud scheduler jobs create http etl-api-bigquery-oneoffice-daily-sync-schedule --location={REGION_SCHEDULER} --schedule='0 1 * * *' --uri='https://{REGION_RUN}-run.googleapis.com/apis/run.googleapis.com/v1/namespaces/{PROJECT_ID}/jobs/etl-api-bigquery-oneoffice-daily-sync:run' --http-method=POST --oauth-service-account-email={SERVICE_ACCOUNT} --time-zone='Asia/Ho_Chi_Minh' --project={PROJECT_ID}"
print("Creating scheduler...")
subprocess.run(cmd + " 2>nul || echo 'Job likely exists'", shell=True)